## 🧠 PDF to LLM Search Pipeline—Step by Step
- **PDF → Extract Text → Chunk → Embedding → Vector DB → Semantic Search → Query to LLM → Final Answer**

In [1]:
print("ok")

ok


In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [5]:
from langchain.document_loaders import PyPDFLoader

In [6]:
%pwd

'f:\\Intership task\\Chatbot\\Question-Answer-Creation-Chatbot\\notebooks'

In [7]:
%cd ..

f:\Intership task\Chatbot\Question-Answer-Creation-Chatbot


In [8]:
%pwd

'f:\\Intership task\\Chatbot\\Question-Answer-Creation-Chatbot'

In [9]:
import PyPDF2

In [10]:
# load dataset(pdf)
file_path = "data/SDG.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [11]:
data

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 2, 'page_label': '3'}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE T

In [12]:
len(data)

24

In [13]:
# Extract all data one by one pages.
question_gen = ""
for page in data:
    question_gen += page.page_content

In [14]:
question_gen

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 yea

In [15]:
from langchain.text_splitter import TokenTextSplitter

In [18]:
# Split the question text into chunks
splitter_question_gen = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size= 10000,
    chunk_overlap = 200
)

chunk_ques_gen = splitter_question_gen.split_text(question_gen)

In [19]:
chunk_ques_gen

['IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 ye

In [20]:
len(chunk_ques_gen)

1

In [21]:
type(chunk_ques_gen[0])

str

In [22]:
from langchain.docstore.document import Document

In [23]:
# Convert each chunk into a Document
document_ques_gen = [Document(page_content=t) for t in chunk_ques_gen]

In [30]:
document_ques_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [31]:
type(document_ques_gen[0])

langchain_core.documents.base.Document

In [24]:
# Split the documents for answer generation
splitter_ans_gen = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=1000,
    chunk_overlap=100
)

document_answer_gen = splitter_ans_gen.split_documents(document_ques_gen)

In [25]:
document_answer_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [26]:
len(document_answer_gen)

4

In [27]:
from langchain.chat_models import ChatOpenAI

In [28]:
# Set up LLM pipeline for question generation
llm_ques_gen_pipeline = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.3
)

C:\Users\User\AppData\Local\Temp\ipykernel_14724\72101767.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_ques_gen_pipeline = ChatOpenAI(


In [32]:
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [33]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [34]:
document_answer_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [35]:
len(document_answer_gen)

4

In [36]:
type(document_answer_gen[0])

langchain_core.documents.base.Document

In [37]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [38]:
from langchain.prompts import PromptTemplate

In [39]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [40]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [41]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [42]:
from langchain.chains.summarize import load_summarize_chain

In [43]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [44]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)

C:\Users\User\AppData\Local\Temp\ipykernel_14724\3836958406.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques = ques_gen_chain.run(document_ques_gen)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough 
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew 
that earthquakes and floods were inevitable, but that the high death 
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders from

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-************************************************************************************************************************ARwA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [45]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [46]:
embeddings = OpenAIEmbeddings()

C:\Users\User\AppData\Local\Temp\ipykernel_14724\2497576997.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [47]:
from langchain.vectorstores import FAISS

In [48]:
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-************************************************************************************************************************ARwA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [49]:
llm_answer_gen = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [50]:
ques

NameError: name 'ques' is not defined

In [ ]:
ques_list = ques.split("\n")

In [ ]:
ques_list

In [ ]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

In [ ]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import load_summary_chain, RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Load the PyPDFLoader (assuming the PDF is already loaded into `question_gen` from previous steps)
question_gen = ""  # Assume this has your loaded text

# Split the question text into chunks
splitter_ques_gen = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=10000,
    chunk_overlap=200
)
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

# Convert each chunk into a Document
document_ques_gen = [Document(page_content=t) for t in chunk_ques_gen]

# Split the documents for answer generation
splitter_ans_gen = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=1000,
    chunk_overlap=100
)
document_answer_gen = splitter_ans_gen.split_documents(document_ques_gen)

# Set up LLM pipeline for question generation
llm_ques_gen_pipeline = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.3
)

# Define refine template and prompt for generating questions
refine_template = """
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
----------
{text}
----------
Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""

REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

# Load the chain to summarize and refine the questions
ques_gen_chain = load_summary_chain(
    llm=llm_ques_gen_pipeline,
    chain_type="refine",
    verbose=True,
    question_prompt=REFINE_PROMPT_QUESTIONS,
    refine_prompt=REFINE_PROMPT_QUESTIONS
)

# Run the chain to generate refined questions
ques = ques_gen_chain.run(document_ques_gen)

# Set up OpenAI embeddings and FAISS vector store
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

# Set up LLM pipeline for answer generation
llm_answer_gen = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo"
)

# Create the RetrievalQA chain to retrieve and generate answers
answer_generation_chain = RetrievalQA.from_chain_type(
    llm=llm_answer_gen,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

# Split the questions into a list
ques_list = ques.split("\n")

# Loop through each question, generate answers, and save them to a file
with open("answers.txt", "a") as f:
    for question in ques_list:
        print("Question: ", question)
        answer = answer_generation_chain.run(question)
        print("Answer: ", answer)
        print("-" * 40)
        
        # Save answer to file
        f.write("Question: " + question + "\n")
        f.write("Answer: " + answer + "\n")
        f.write("-" * 40 + "\n")
